In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests as req
from census import Census
import json
import csv



# Census API Key
#from config import api_key
c = Census("49f2db3e7faca8ecd92103d5bd5c5c765442d598", year=2016)


In [45]:
# Logic:
#Get the cites to query
#Get the census fields to query

sourceFile = pd.ExcelFile('Project1_AmazonSites.xlsx')

SitesDF=sourceFile.parse('AmazonSites') 

CitiesDf=sourceFile.parse('AmazonCities') 
CitiesDf=CitiesDf[['PlaceCode','StateCode']]

CensusFieldsDF=sourceFile.parse('CensusFields')
CensusFieldsDF = CensusFieldsDF[["Datafield","CensusCode"]]

#Converting the CensusFieldsDF into a dictionary.
dictCensusFields = CensusFieldsDF.set_index('CensusCode').to_dict()
dictCensusFields = dictCensusFields['Datafield']

print(dictCensusFields)
#CitiesDf.head()

{'B01002_001E': 'Median Age', 'B25077_001E': 'Median Home Value', 'B25064_001E': 'Median Rent', 'B23025_002E': 'LaborForce', 'B23025_004E': 'Emp_Total', 'B23025_003E': 'Emp_civilian_labor_force', 'C24010_041E': 'Emp_Mgment_Occp_Female', 'C24010_005E': 'Emp_Mgment_Occp_Male', 'B15003_021E': 'Ed_Associates', 'B15003_022E': 'Ed_Bachelors', 'B15003_025E': 'Ed_Doctorate', 'B15003_018E': 'Ed_GED', 'B15003_017E': 'Ed_HighSchool', 'B15003_023E': 'Ed_Masters', 'B15003_024E': 'Ed_Professional', 'B01003_001E': 'Total Population', 'B02001_004E': 'Pop_American_Indian', 'B02001_005E': 'Population_Asian', 'B02001_003E': 'Pop_Black', 'B03001_003E': 'Pop_Hispanic_Origin', 'B02001_006E': 'Pop_Native_Hawaiian', 'B02001_007E': 'Pop_Other', 'B02001_008E': 'Pop_two_or_more_races', 'B02001_002E': 'Pop_White', 'B08136_001E': 'Commute_Time', 'B08136_004E': 'Commute_Time_Carpool', 'B08136_012E': 'Commute_Time_Other', 'B08136_007E': 'Commute_Time_Public_Transport', 'B08136_003E': 'Commute_Time_Solo_Automobile'}


In [48]:

# Build your Census API call.
arrCensusFields = ['NAME']

for k in dictCensusFields.keys():
    arrCensusFields.append(k)
    
censusDF= pd.DataFrame()

for city in CitiesDf.values:
    cityCode = str(city[0])
    stateCode = str(city[1])

    if len(cityCode)==4:
        cityCode = "0"+cityCode
    if len(stateCode)==1:
        stateCode = "0"+stateCode
        
    varGeo = {'for':'place:'+ cityCode,'in':'state:'+stateCode}
    temp1 = c.acs5.get(arrCensusFields, varGeo)
  
    tempDF = pd.DataFrame(temp1[0], index = [0])    
    censusDF = censusDF.append(tempDF,ignore_index = True)


censusDF = censusDF.rename(columns=dictCensusFields)
censusDF.to_csv("Cities_Demographics.csv")
censusDF.head()

,Median Age,Total Population,Pop_White,Pop_Black,Pop_American_Indian,Population_Asian,Pop_Native_Hawaiian,Pop_Other,Pop_two_or_more_races,Pop_Hispanic_Origin,...,LaborForce,Emp_civilian_labor_force,Emp_Total,Median Rent,Median Home Value,Emp_Mgment_Occp_Male,Emp_Mgment_Occp_Female,NAME,place,state
0,33.8,659009.0,266035.0,318598.0,2174.0,24036.0,271.0,29650.0,18245.0,69106.0,...,385056.0,381551.0,348225.0,1362.0,506100.0,29460.0,26678.0,"Washington city, District of Columbia",50000,11
1,35.9,8461961.0,3644982.0,2064927.0,33388.0,1159883.0,3995.0,1277265.0,277521.0,2457945.0,...,4369307.0,4367089.0,3991617.0,1294.0,508900.0,219793.0,175988.0,"New York city, New York",51000,36
2,31.7,658279.0,348906.0,167282.0,2577.0,61490.0,122.0,45492.0,32410.0,125087.0,...,385571.0,384980.0,354352.0,1369.0,423200.0,18783.0,19614.0,"Boston city, Massachusetts",07000,25
3,33.9,2714017.0,1321324.0,839917.0,7818.0,165229.0,906.0,310466.0,68357.0,790548.0,...,1449953.0,1449413.0,1290814.0,987.0,225200.0,67754.0,57280.0,"Chicago city, Illinois",14000,17
4,35.0,3918872.0,2055223.0,353023.0,26660.0,452853.0,6716.0,884822.0,139575.0,1905577.0,...,2097465.0,2096761.0,1903882.0,1241.0,496300.0,102514.0,73709.0,"Los Angeles city, California",44000,06
